In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
import sys
from neuralop.models import FNO
from neuralop import Trainer
from neuralop.training import AdamW
from neuralop.data.datasets import load_darcy_flow_small
from neuralop.utils import count_model_params
from neuralop import LpLoss, H1Loss


In [2]:
from torch import optim

In [3]:
datastore = h5py.File("C:/Users/arnab/OneDrive/Desktop/Study material/summer 24/Research/New equations/datasets/trials_many_alldata_new_eqution2.h5", 'r')
data_I = np.array(np.array(datastore["I"]))

data_c = np.array(np.array(datastore["c"]))
data_h = np.array(np.array(datastore["h"]))
t = np.linspace(0, 1, 601).reshape(601, 1)

In [4]:
train_label, test_label = train_test_split(range(len(data_I)), test_size=0.25, random_state=42)

data_i_train = data_I[train_label,:]
data_i_test = data_I[test_label,:]

data_c_train = data_c[train_label,:]
data_c_test = data_c[test_label,:]

data_h_train = data_h[train_label,:]
data_h_test = data_h[test_label,:]

In [5]:
data_i_train = torch.tensor(data_i_train, dtype=torch.float32).unsqueeze(1)
data_i_test = torch.tensor(data_i_test, dtype=torch.float32).unsqueeze(1)

data_c_train = torch.tensor(data_c_train, dtype=torch.float32).unsqueeze(1)
data_c_test = torch.tensor(data_c_test, dtype=torch.float32).unsqueeze(1)

data_h_train = torch.tensor(data_h_train, dtype=torch.float32).unsqueeze(1)
data_h_test = torch.tensor(data_h_test, dtype=torch.float32).unsqueeze(1)

In [10]:
train_dataset_c = TensorDataset(data_i_train, data_c_train)
train_dataloader_c = DataLoader(train_dataset_c, batch_size=1024, shuffle=True)

test_dataset_c = TensorDataset(data_i_test, data_c_test)
test_dataloader_c = DataLoader(test_dataset_c, batch_size=1024, shuffle=True)

In [11]:
device = 'cpu'

model_c = FNO(
    n_modes=(16,),         
    hidden_channels=64,    
    in_channels=1,        
    out_channels=1         
)
model_c = model_c.to(device)

n_params = count_model_params(model_c)
print(f'\nOur model has {n_params} parameters.')
sys.stdout.flush()



Our model has 345665 parameters.


In [12]:
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model_c.parameters(), lr=0.001)

In [13]:
num_epochs = 10
# batch_size = 32

for epoch in range(num_epochs):
    model_c.train()  
    epoch_loss = 0.0

    for inputs, targets in train_dataloader_c:
        optimizer.zero_grad() 
        predictions = model_c(inputs)  
        loss = criterion(predictions, targets)  
        loss.backward()  
        optimizer.step() 
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_dataloader_c)}")

Epoch 1/10, Loss: 8.531809417212882
Epoch 2/10, Loss: 6.622102115212417
Epoch 3/10, Loss: 4.272974793503924
Epoch 4/10, Loss: 2.062503628614472
Epoch 5/10, Loss: 1.569042022635297
Epoch 6/10, Loss: 1.2395629773779613
Epoch 7/10, Loss: 0.9944444996554677
Epoch 8/10, Loss: 0.7527750745052244
Epoch 9/10, Loss: 0.8882613265659751
Epoch 10/10, Loss: 0.6493649595394367


In [14]:
model_c.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_dataloader_c:
        predictions = model_c(inputs)
        loss = criterion(predictions, targets)
        test_loss += loss.item()
print(f"Test Loss: {test_loss / len(test_dataloader_c)}")

Test Loss: 0.5073423566562789


In [15]:
train_dataset_h = TensorDataset(data_i_train, data_h_train)
train_dataloader_h = DataLoader(train_dataset_h, batch_size=1024, shuffle=True)

test_dataset_h = TensorDataset(data_i_test, data_h_test)
test_dataloader_h = DataLoader(test_dataset_h, batch_size=1024, shuffle=True)

In [16]:
model_h = FNO(
    n_modes=(16,),         
    hidden_channels=40,    
    in_channels=1,        
    out_channels=1         
)
model_h = model_c.to(device)

n_params = count_model_params(model_h)
print(f'\nOur model has {n_params} parameters.')
sys.stdout.flush()



Our model has 345665 parameters.


In [17]:
num_epochs = 5
# batch_size = 32

for epoch in range(num_epochs):
    model_h.train()  
    epoch_loss = 0.0

    for inputs, targets in train_dataloader_h:
        optimizer.zero_grad() 
        predictions = model_h(inputs)  
        loss = criterion(predictions, targets)  
        loss.backward()  
        optimizer.step() 
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_dataloader_h)}")

Epoch 1/5, Loss: 0.31314539268794583
Epoch 2/5, Loss: 0.06267611069105021
Epoch 3/5, Loss: 0.061953247184070144
Epoch 4/5, Loss: 0.061158889086871615
Epoch 5/5, Loss: 0.06013041021438634


In [18]:
model_h.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_dataloader_h:
        predictions = model_h(inputs)
        loss = criterion(predictions, targets)
        test_loss += loss.item()
print(f"Test Loss: {test_loss / len(test_dataloader_h)}")

Test Loss: 0.06038714985230139
